<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Tutorial on generating an explanation for a text-based model on Watson OpenScale

This notebook includes steps for creating a text-based watson-machine-learning model, creating a subscription, configuring explainability, and finally generating an explanation for a transaction.

### Contents
- [1. Setup](#setup)
- [2. Creating and deploying a text-based model](#deploy)
- [3. Subscriptions](#subscription)
- [4. Explainability](#explainability)

**Note**: This notebook works correctly with kernel `Python 3.10.x` with pyspark 3.3.x.

<a id="setup"></a>
## 1. Setup

### 1.1 Install Watson OpenScale and WML packages

In [59]:
!pip install --upgrade ibm-watson-openscale --no-cache --user| tail -n 1

In [60]:
!pip install --upgrade ibm-watson-machine-learning --no-cache --user| tail -n 1

Note: Restart the kernel to assure the new libraries are being used.

### 1.2 Configure credentials

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [61]:
CLOUD_API_KEY = "..."
IAM_URL="https://iam.us-south.bluemix.net/oidc/token"

In [62]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": CLOUD_API_KEY
}

## 2. Creating and deploying a text-based model <a id="deploy"></a>

The dataset used is the UCI-ML SMS Spam Collection Dataset which can be found here: https://archive.ics.uci.edu/ml/machine-learning-databases/00228/. It is a binary classification dataset with the labels being 'ham' and 'spam'.

### 2.1 Loading the training data

In [63]:
!rm -rf SMSSpam.csv
!wget 'https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/spam_detection/SMSSpam.csv'

--2023-06-13 10:26:28--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/spam_detection/SMSSpam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480803 (470K) [text/plain]
Saving to: ‘SMSSpam.csv’

SMSSpam.csv         100%[===================>] 469.53K  --.-KB/s    in 0.009s  

2023-06-13 10:26:28 (50.7 MB/s) - ‘SMSSpam.csv’ saved [480803/480803]



In [64]:
# The training data is downloaded and saved as 'SMSSpam.csv' in this step from public link

# !pip install pandas
# !rm smsspamcollection.zip
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
# !unzip smsspamcollection.zip
#pd.read_csv("smsspamcollection.zip",sep="\t",header=None, encoding="utf-8").to_csv("SMSSpam.csv", header=["label", "text"], sep=",", index=False)

# !rm SMSSpamCollection
# !rm readme
# !rm smsspamcollection.zip

### 2.2 Creating a model

**Note**: Skip the pyspark install step below if you are using a Spark kernel on Watson Studio.

In [65]:
!pip show pyspark

Name: pyspark
Version: 3.3.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /home/wsuser/.local/lib/python3.10/site-packages
Requires: py4j
Required-by: 


In [66]:
!pip install --upgrade pyspark==3.3.0 --user

**Note**: When running this notebook locally, If the `SparkSession` import fails below, set 'SPARK_HOME' environment variable with the path to `pyspark` installation.

In [67]:
!export SPARK_HOME=/home/wsuser/.local/lib/python3.10/site-packages/pyspark
!echo $SPARK_HOME

In [68]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv(path="SMSSpam.csv", header=True, multiLine=True, escape='"')
df.show(5, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|text                                                                                                                                                       |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                            |
|ham  |Ok lar... Joking wif u oni...                                                                                                                              |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's|
|ham  |U dun say

In [69]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=12345)
print("Total count of data set: {}".format(df.count()))
print("Total count of training data set: {}".format(train_df.count()))
print("Total count of test data set: {}".format(test_df.count()))

Total count of data set: 5572
Total count of training data set: 4454
Total count of test data set: 1118


In [70]:
!pip install nltk
from pyspark.ml.feature import StringIndexer, IndexToString, CountVectorizer, Tokenizer, IDF, StopWordsRemover
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

stringIndexer_label = StringIndexer(inputCol="label", outputCol="label_ix").fit(df)
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopword_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words").setStopWords(stop_words)
count = CountVectorizer(inputCol="filtered_words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
nb = GBTClassifier(labelCol="label_ix")
labelConverter = IndexToString(inputCol="prediction", outputCol="predictionLabel", labels=stringIndexer_label.labels)

[nltk_data] Downloading package punkt to /home/wsuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [71]:
pipeline = Pipeline(stages=[stringIndexer_label, tokenizer, stopword_remover, count, idf, nb, labelConverter])
model = pipeline.fit(train_df)
predictions = model.transform(test_df)
evaluator = BinaryClassificationEvaluator(labelCol="label_ix", rawPredictionCol="prediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predictions)

print("Area under ROC curve = %g" % auc)

Area under ROC curve = 0.856224


In [72]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.308'

In [73]:
wml_client.spaces.list(limit=10)

------------------------------------  -------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                 CREATED
06c034ae-6f50-44dd-8339-6306e7a440f1  openscale-express-path-preprod-498a4f21-085b-4d5e-a41f-00ba872e60cc  2023-06-13T10:16:31.848Z
1498b034-5c44-4cdd-a7fb-d8f14078ec9c  openscale-express-path-498a4f21-085b-4d5e-a41f-00ba872e60cc          2023-06-13T10:16:11.164Z
9510bdc6-114e-4ad0-84f4-bdcb4d89e253  AutoAI                                                               2023-04-27T18:28:18.112Z
------------------------------------  -------------------------------------------------------------------  ------------------------


,ID,NAME,CREATED
0,06c034ae-6f50-44dd-8339-6306e7a440f1,openscale-express-path-preprod-498a4f21-085b-4...,2023-06-13T10:16:31.848Z
1,1498b034-5c44-4cdd-a7fb-d8f14078ec9c,openscale-express-path-498a4f21-085b-4d5e-a41f...,2023-06-13T10:16:11.164Z
2,9510bdc6-114e-4ad0-84f4-bdcb4d89e253,AutoAI,2023-04-27T18:28:18.112Z


In [74]:
WML_SPACE_ID='9510bdc6-114e-4ad0-84f4-bdcb4d89e253' # use space id here
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

In [75]:
MODEL_NAME = "Imdb - P16 Snap Boosting Machine Classifier"

In [76]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("spark-mllib_3.3")
print("Software Specification ID: {}".format(software_spec_uid))
model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.TYPE: "mllib_3.3",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "label",
    }

Software Specification ID: d11f2434-4fc7-58b7-8a62-755da64fdaf8


In [77]:
print("Storing model ...")
published_model_details = wml_client.repository.store_model(
    model=model, 
    meta_props=model_props, 
    training_data=train_df, 
    pipeline=pipeline)

model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: 7e1dc8a0-7cc7-4852-82bf-2e631939617f


### 2.3 Deploying the model

In [78]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(MODEL_NAME + " deployment"),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_id(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))



#######################################################################################

Synchronous deployment creation for uid: '7e1dc8a0-7cc7-4852-82bf-2e631939617f' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='ac59a7c4-7783-49fa-97d3-d3078bfdfa40'
------------------------------------------------------------------------------------------------


Scoring URL:https://us-south.ml.cloud.ibm.com/ml/v4/deployments/ac59a7c4-7783-49fa-97d3-d3078bfdfa40/predictions
Model id: 7e1dc8a0-7cc7-4852-82bf-2e631939617f
Deployment id: ac59a7c4-7783-49fa-97d3-d3078bfdfa40


## 3. Subscriptions <a id="subscription"></a>

### 3.1 Configuring OS

In [79]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY, url = "https://iam.cloud.ibm.com/identity/token")
#authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment this line if using IAM token to authenticate
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.31'

**Note**: Please re-run the above cell if it doesn't work the first time.

In [80]:
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}
DB_CREDENTIALS = None
KEEP_MY_INTERNAL_POSTGRES = True

In [81]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 498a4f21-085b-4d5e-a41f-00ba872e60cc


In [84]:
SERVICE_PROVIDER_NAME = "WOS ExpressPath WML production binding"
SERVICE_PROVIDER_DESCRIPTION = "Give a name here"

In [85]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    print(service_instance_name)
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

WOS ExpressPath WML pre_production binding
WOS ExpressPath WML production binding
Deleted existing service_provider for WML instance: 97567d9d-6572-43d0-8c13-98547cd1f76f


In [86]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WMLCredentialsCloud(
            apikey=CLOUD_API_KEY,      ## use `apikey=IAM_TOKEN` if using IAM_TOKEN to initiate client
            url=WML_CREDENTIALS["url"],
            instance_id=None
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 8e355a06-7007-48b4-b253-e3d46e4b534b 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [91]:
asset_deployment_details_list = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id, deployment_space_id = WML_SPACE_ID).result['resources']
print([asset['entity']['name'] for asset in asset_deployment_details_list])
DEPLOYMENT_NAME='Imdb - P16 Snap Boosting Machine Classifier deployment' # use the model name here 
asset_deployment_details = [asset for asset in asset_deployment_details_list if asset['entity']["name"]==DEPLOYMENT_NAME]

if len(asset_deployment_details)>0:
    [asset_deployment_details] = asset_deployment_details
else:
    raise ValueError('deployment with name "{}" not found.'.format(DEPLOYMENT_NAME))
asset_deployment_details

['AutoAI deployment', 'Imdb - P16 Snap Boosting Machine Classifier deployment']


{'metadata': {'guid': 'ac59a7c4-7783-49fa-97d3-d3078bfdfa40',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/ac59a7c4-7783-49fa-97d3-d3078bfdfa40?space_id=9510bdc6-114e-4ad0-84f4-bdcb4d89e253',
  'created_at': '2023-06-13T10:31:11.876Z',
  'modified_at': '2023-06-13T10:31:11.876Z'},
 'entity': {'name': 'Imdb - P16 Snap Boosting Machine Classifier deployment',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/ac59a7c4-7783-49fa-97d3-d3078bfdfa40/predictions'},
  'asset': {},
  'asset_properties': {}}}

In [92]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,deployment_space_id=WML_SPACE_ID)
model_asset_details_from_deployment

{'metadata': {'guid': 'ac59a7c4-7783-49fa-97d3-d3078bfdfa40',
  'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/ac59a7c4-7783-49fa-97d3-d3078bfdfa40?space_id=9510bdc6-114e-4ad0-84f4-bdcb4d89e253',
  'created_at': '2023-06-13T10:31:11.876Z',
  'modified_at': '2023-06-13T10:31:11.876Z'},
 'entity': {'name': 'Imdb - P16 Snap Boosting Machine Classifier deployment',
  'type': 'online',
  'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/ac59a7c4-7783-49fa-97d3-d3078bfdfa40/predictions'},
  'asset': {'asset_id': '7e1dc8a0-7cc7-4852-82bf-2e631939617f',
   'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/models/7e1dc8a0-7cc7-4852-82bf-2e631939617f?space_id=9510bdc6-114e-4ad0-84f4-bdcb4d89e253&version=2020-06-12',
   'name': 'Imdb - P16 Snap Boosting Machine Classifier',
   'asset_type': 'model',
   'created_at': '2023-06-13T10:31:00.191Z',
   'modified_at': '2023-06-13T10:31:10.172Z'},
  'asset_properties': {'model_type': 'mllib_3.3',
   'runtime_envi

### 3.2 Subscribe the asset

In [93]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [94]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest

In [95]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.UNSTRUCTURED_TEXT,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['metadata']['url'],
            scoring_endpoint=ScoringEndpointRequest(url=scoring_url) # scoring model without shadow deployment
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='label',
            probability_fields=['probability'],
            prediction_field='predictionLabel',
            feature_fields = ["text"],
            categorical_fields = ["text"],
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result
subscription_id = subscription_details.metadata.id
subscription_id

'f78c2891-7d0e-4f79-abf8-44287938077c'

In [96]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  69136898-43b8-49de-96ef-066093aeb56c


### 3.3 Get subscription

In [97]:
wos_client.subscriptions.show()

7e1dc8a0-7cc7-4852-82bf-2e631939617f,Imdb - P16 Snap Boosting Machine Classifier,498a4f21-085b-4d5e-a41f-00ba872e60cc,ac59a7c4-7783-49fa-97d3-d3078bfdfa40,Imdb - P16 Snap Boosting Machine Classifier deployment,8e355a06-7007-48b4-b253-e3d46e4b534b,active,2023-06-13 10:35:12.869000+00:00,f78c2891-7d0e-4f79-abf8-44287938077c
457aaf84-1b3e-43ea-83d0-6452480ff393,GermanCreditRiskModelPreProd,498a4f21-085b-4d5e-a41f-00ba872e60cc,f8c9381f-b6e7-415e-8267-bc106ce26fd0,GermanCreditRiskModelPreProd,f4b2d06b-0c87-4dd1-aec9-53c65e4d7ec0,active,2023-06-13 10:19:04.538000+00:00,ff514e86-e745-4c27-b607-ec46ac8c7859
2fcea401-a083-4c50-b22e-280f02d36a66,GermanCreditRiskModelChallenger,498a4f21-085b-4d5e-a41f-00ba872e60cc,04aaec62-3fdb-4977-8d2b-61d00ca46756,GermanCreditRiskModelChallenger,f4b2d06b-0c87-4dd1-aec9-53c65e4d7ec0,active,2023-06-13 10:17:41.570000+00:00,4cdb9796-1d45-4b1e-ab58-47268d785b50


In [98]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

{'metadata': {'id': 'f78c2891-7d0e-4f79-abf8-44287938077c',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/5e896714a2f24ca7bdf58aaf328e8bd8:498a4f21-085b-4d5e-a41f-00ba872e60cc:subscription:f78c2891-7d0e-4f79-abf8-44287938077c',
  'url': '/v2/subscriptions/f78c2891-7d0e-4f79-abf8-44287938077c',
  'created_at': '2023-06-13T10:35:12.869000Z',
  'created_by': 'IBMid-6660020YC8'},
 'entity': {'data_mart_id': '498a4f21-085b-4d5e-a41f-00ba872e60cc',
  'service_provider_id': '8e355a06-7007-48b4-b253-e3d46e4b534b',
  'asset': {'asset_id': '7e1dc8a0-7cc7-4852-82bf-2e631939617f',
   'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/models/7e1dc8a0-7cc7-4852-82bf-2e631939617f?space_id=9510bdc6-114e-4ad0-84f4-bdcb4d89e253&version=2020-06-12',
   'name': 'Imdb - P16 Snap Boosting Machine Classifier',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'unstructured_text'},
  'asset_properties': {'training_data_schema': {'type': 'struct',
    'fields': [{'metadata': {'

### 3.4 Score the model and get transaction-id

In [99]:
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

In [105]:
text = "Hey, where are you? I'm waiting for you"
payload = {"input_data": [{"fields": ["text"], "values": [[text]]}]}

response = wml_client.deployments.score(deployment_uid,payload)
print(response)

{'predictions': [{'fields': ['text', 'label_ix', 'words', 'filtered_words', 'rawFeatures', 'features', 'rawPrediction', 'probability', 'prediction', 'predictionLabel'], 'values': [["Hey, where are you? I'm waiting for you", 0.0, ['hey,', 'where', 'are', 'you?', "i'm", 'waiting', 'for', 'you'], ['hey,', 'you?', "i'm", 'waiting'], [11799, [5, 148, 358, 1300], [1.0, 1.0, 1.0, 1.0]], [11799, [5, 148, 358, 1300], [2.7777648337175713, 4.6882102732006015, 5.406050066350919, 6.455872190849597]], [1.3657066884390638, -1.3657066884390638], [0.938855024215716, 0.06114497578428402], 0.0, 'ham']]}]}


In [106]:
wos_client.data_sets.get_records_count(payload_data_set_id)

1

## 4. Explainability

### 4.1 Configure Explainability

In [107]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

ApiException: Error: Resource could not be created because of conflict with existing resource `0df2cea1-1795-490c-9d1b-e481231c33ba` of type `monitor_instance`, Code: 409 , X-global-transaction-id: YWZmYzMxZGMtZTA3Ny00YzhiLWFmY2UtYzkyODk2MmFjNTVk

### 4.2 Get explanation for the transaction

In [108]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)

Running explanations on scoring IDs: ['b42c0c91ee06eb2c60ead3dc5d6f257f-1']
{
  "metadata": {
    "explanation_task_ids": [
      "d9618f70-1811-4b59-8ee0-93e1deb90741"
    ],
    "created_by": "IBMid-6660020YC8",
    "created_at": "2023-06-13T11:18:26.179128Z"
  }
}


In [109]:
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id, subscription_id=subscription_id).result.to_dict()

{'metadata': {'explanation_task_id': 'd9618f70-1811-4b59-8ee0-93e1deb90741',
  'created_by': 'IBMid-6660020YC8',
  'created_at': '2023-06-13T11:18:26.179128Z'},
 'entity': {'status': {'state': 'in_progress'},
  'asset': {'id': '7e1dc8a0-7cc7-4852-82bf-2e631939617f',
   'name': 'Imdb - P16 Snap Boosting Machine Classifier',
   'input_data_type': 'unstructured_text',
   'problem_type': 'binary',
   'deployment': {'id': 'ac59a7c4-7783-49fa-97d3-d3078bfdfa40',
    'name': 'Imdb - P16 Snap Boosting Machine Classifier deployment'}},
  'scoring_id': 'b42c0c91ee06eb2c60ead3dc5d6f257f-1'}}